# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import graphviz

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [122]:
X_train = pd.read_csv('./Atividade 1/UCI HAR Dataset/train/X_train.txt',
                     sep=r'\s+', header=None)
X_test = pd.read_csv('./Atividade 1/UCI HAR Dataset/test/X_test.txt',
                    sep=r'\s+', header=None)

y_train = pd.read_csv('./Atividade 1/UCI HAR Dataset/train/y_train.txt',
                      sep=r'\s+',
                      header=None)
y_test = pd.read_csv('./Atividade 1/UCI HAR Dataset/test/y_test.txt',
                      sep=r'\s+',
                      header=None)

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [124]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.25, random_state=1729)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [126]:
clf = DecisionTreeClassifier(max_depth=4, random_state=2360873)
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

clfs = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873,
                                ccp_alpha=ccp_alpha).fit(X_train, y_train)
    clfs.append(clf)

In [128]:
importante = pd.Series(clf.feature_importances_, index=X_train.columns)
importante_ordenado = importante.sort_values(ascending=False)
top_3 = importante_ordenado.head(3)
print(top_3)

0      0.0
377    0.0
371    0.0
dtype: float64


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [184]:
%%time
var_top3 = top_3.index.tolist()
X_train_top3 = X_train[var_top3]
X_test_top3 = X_test[var_top3]

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train_top3, y_train, 
                                                                test_size=0.25, 
                                                                random_state=1729)

modelo = DecisionTreeClassifier(random_state=42)
caminho = modelo.cost_complexity_pruning_path(X_valid, y_valid)
ccp_alphas = caminho.ccp_alphas

clfs0 = []

for ccp_alpha in ccp_alphas:
    clf1 = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha).fit(X_valid, y_valid)
    clfs0.append(clf1)

CPU times: total: 1.92 s
Wall time: 1.89 s


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [186]:
train_scores = [clf.score(X_train_part, y_train_part) for clf in clfs0]
valid_scores = [clf.score(X_valid, y_valid) for clf in clfs0]

In [188]:
best_tree = len(valid_scores) - valid_scores[::-1].index(max(valid_scores)) - 1
melhor_arvore = clfs0[best_tree]

print(f'A melhor acurácia é {valid_scores[best_tree]*100:.2f}')

valid_tree

A melhor acurácia é 100.00


DecisionTreeClassifier(ccp_alpha=0.0023266611965221534, random_state=42)